In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6853566342880726112
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
locality {
  bus_id: 1
}
incarnation: 787013855824534352
physical_device_desc: "device: 0, name: METAL, pci bus id: <undefined>"
xla_global_id: -1
]


2023-07-08 12:11:04.024294: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-07-08 12:11:04.024309: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-07-08 12:11:04.024312: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-07-08 12:11:04.024514: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-08 12:11:04.024526: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


2023-07-08 12:11:08.657977: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-08 12:11:08.658070: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing, utils
import re
import yaml
import os


def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list, vocabulary


def data_generator(encoder_input_data, decoder_input_data, decoder_output_data, batch_size):
    num_samples = encoder_input_data.shape[0]
    steps_per_epoch = num_samples // batch_size

    while True:
        for step in range(steps_per_epoch):
            batch_start = step * batch_size
            batch_end = (step + 1) * batch_size

            batch_encoder_input = encoder_input_data[batch_start:batch_end]
            batch_decoder_input = decoder_input_data[batch_start:batch_end]
            batch_decoder_output = decoder_output_data[batch_start:batch_end]

            yield [batch_encoder_input, batch_decoder_input], batch_decoder_output


dir_path = 'archive'
files_list = os.listdir(dir_path + os.sep)
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath, 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2:
            questions.append(con[0])
            replies = con[1:]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con) > 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)):
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index) + 1

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions, maxlen=maxlen_questions, padding='post')
encoder_input_data = np.array(padded_questions)

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_input_data = np.array(padded_answers)

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)):
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers, maxlen=maxlen_answers, padding='post')
decoder_output_data = np.array(padded_answers)

# Embedding, LSTM and Dense layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200, mask_zero=True)(encoder_inputs)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(200, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers,))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200, mask_zero=True)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE, activation=tf.keras.activations.softmax)
output = decoder_dense(decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

# Define the batch size
batch_size = 16

# Create the data generator
train_generator = data_generator(encoder_input_data, decoder_input_data, decoder_output_data, batch_size)

# Train the model using the generator
steps_per_epoch = len(tokenized_questions) // batch_size
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=300)


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 38)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 563)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 38, 200)              1430000   ['input_5[0][0]']             
                                                                                                  
 embedding_3 (Embedding)     (None, 563, 200)             1430000   ['input_6[0][0]']             
                                                                                            

2023-07-08 13:05:26.270861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-08 13:05:26.630279: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-08 13:05:26.813665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-08 13:05:28.149176: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  1/147 [..............................] - ETA: 15:03 - loss: 8.8748 - accuracy: 0.0000e+00

2023-07-08 13:05:30.851373: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


147/147 [==============================] - 46s 271ms/step - loss: 7.1508 - accuracy: 0.0509
Epoch 2/300
147/147 [==============================] - 38s 262ms/step - loss: 6.3871 - accuracy: 0.0671
Epoch 3/300
147/147 [==============================] - 39s 268ms/step - loss: 6.1779 - accuracy: 0.0903
Epoch 4/300
147/147 [==============================] - 41s 278ms/step - loss: 6.0124 - accuracy: 0.0989
Epoch 5/300
147/147 [==============================] - 39s 266ms/step - loss: 5.8552 - accuracy: 0.1074
Epoch 6/300
147/147 [==============================] - 39s 266ms/step - loss: 5.6971 - accuracy: 0.1208
Epoch 7/300
147/147 [==============================] - 39s 266ms/step - loss: 5.5396 - accuracy: 0.1351
Epoch 8/300
147/147 [==============================] - 39s 265ms/step - loss: 5.3790 - accuracy: 0.1505
Epoch 9/300
147/147 [==============================] - 40s 271ms/step - loss: 5.2221 - accuracy: 0.1619
Epoch 10/300
147/147 [==============================] - 39s 267ms/step - los

147/147 [==============================] - 44s 300ms/step - loss: 1.0978 - accuracy: 0.6632
Epoch 80/300
147/147 [==============================] - 43s 293ms/step - loss: 1.0793 - accuracy: 0.6686
Epoch 81/300
147/147 [==============================] - 44s 297ms/step - loss: 1.0606 - accuracy: 0.6741
Epoch 82/300
147/147 [==============================] - 44s 298ms/step - loss: 1.0405 - accuracy: 0.6806
Epoch 83/300
147/147 [==============================] - 45s 309ms/step - loss: 1.0273 - accuracy: 0.6831
Epoch 84/300
147/147 [==============================] - 44s 300ms/step - loss: 1.0157 - accuracy: 0.6880
Epoch 85/300
147/147 [==============================] - 46s 312ms/step - loss: 0.9976 - accuracy: 0.6927
Epoch 86/300
147/147 [==============================] - 44s 297ms/step - loss: 0.9821 - accuracy: 0.6974
Epoch 87/300
147/147 [==============================] - 43s 294ms/step - loss: 0.9694 - accuracy: 0.7013
Epoch 88/300
147/147 [==============================] - 43s 294ms/st

147/147 [==============================] - 43s 295ms/step - loss: 0.4952 - accuracy: 0.8731
Epoch 157/300
147/147 [==============================] - 44s 300ms/step - loss: 0.4918 - accuracy: 0.8763
Epoch 158/300
147/147 [==============================] - 43s 292ms/step - loss: 0.4961 - accuracy: 0.8727
Epoch 159/300
147/147 [==============================] - 43s 293ms/step - loss: 0.4920 - accuracy: 0.8759
Epoch 160/300
147/147 [==============================] - 43s 290ms/step - loss: 0.4842 - accuracy: 0.8792
Epoch 161/300
147/147 [==============================] - 44s 297ms/step - loss: 0.4782 - accuracy: 0.8821
Epoch 162/300
147/147 [==============================] - 43s 292ms/step - loss: 0.4721 - accuracy: 0.8863
Epoch 163/300
147/147 [==============================] - 43s 293ms/step - loss: 0.4663 - accuracy: 0.8891
Epoch 164/300
147/147 [==============================] - 43s 292ms/step - loss: 0.4608 - accuracy: 0.8912
Epoch 165/300
147/147 [==============================] - 41s

147/147 [==============================] - 47s 319ms/step - loss: 0.4015 - accuracy: 0.9196
Epoch 234/300
147/147 [==============================] - 46s 310ms/step - loss: 0.3929 - accuracy: 0.9264
Epoch 235/300
147/147 [==============================] - 49s 331ms/step - loss: 0.3823 - accuracy: 0.9335
Epoch 236/300
147/147 [==============================] - 46s 314ms/step - loss: 0.3723 - accuracy: 0.9380
Epoch 237/300
147/147 [==============================] - 46s 311ms/step - loss: 0.3653 - accuracy: 0.9416
Epoch 238/300
147/147 [==============================] - 46s 310ms/step - loss: 0.3603 - accuracy: 0.9439
Epoch 239/300
147/147 [==============================] - 46s 310ms/step - loss: 0.3581 - accuracy: 0.9452
Epoch 240/300
147/147 [==============================] - 46s 312ms/step - loss: 0.3562 - accuracy: 0.9458
Epoch 241/300
147/147 [==============================] - 46s 312ms/step - loss: 0.3556 - accuracy: 0.9464
Epoch 242/300
147/147 [==============================] - 47s

In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models , preprocessing, utils
import re

import yaml
import os

    

dir_path = 'archive'
files_list = [file for file in os.listdir(dir_path) if not file.startswith('.DS_Store')]
questions, answers = [], []

for filepath in files_list:
    file_ = open(dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(file_)
    conversations = docs['conversations']
    for con in conversations:
        if len(con) > 2 :
            questions.append(con[0])
            replies = con[1 :]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append(ans)
        elif len(con)> 1:
            questions.append(con[0])
            answers.append(con[1])

answers_with_tags = []
for i in range(len(answers)):
    if type(answers[i]) == str:
        answers_with_tags.append(answers[i])
    else:
        questions.pop(i)

answers = []
for i in range(len(answers_with_tags)) :
    answers.append('<START> ' + answers_with_tags[i] + ' <END>')

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)
VOCAB_SIZE = len(tokenizer.word_index)+1

from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
    return tokens_list , vocabulary

# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences(questions)
maxlen_questions = max([len(x) for x in tokenized_questions])
padded_questions = preprocessing.sequence.pad_sequences(tokenized_questions , maxlen=maxlen_questions , padding='post')
encoder_input_data = np.array(padded_questions)

encoder_input_data.shape

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
maxlen_answers = max([len(x) for x in tokenized_answers])
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
decoder_input_data = np.array(padded_answers)

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences(answers)
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences(tokenized_answers , maxlen=maxlen_answers , padding='post')
onehot_answers = utils.to_categorical(padded_answers , VOCAB_SIZE)
decoder_output_data = np.array(onehot_answers)

# Embedding, LSTM and Desne layers
encoder_inputs = tf.keras.layers.Input(shape=(maxlen_questions ,))
encoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM(200 , return_state=True)(encoder_embedding)
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=(maxlen_answers , ))
decoder_embedding = tf.keras.layers.Embedding(VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(200 , return_state=True , return_sequences=True)
decoder_outputs , _ , _ = decoder_lstm (decoder_embedding , initial_state=encoder_states)


decoder_dense = tf.keras.layers.Dense(VOCAB_SIZE , activation=tf.keras.activations.softmax) 
output = decoder_dense (decoder_outputs)

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

    

model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=16, epochs=10)

 

2023-07-03 16:22:10.929243: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-07-03 16:22:10.929261: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-07-03 16:22:10.929264: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-07-03 16:22:10.929291: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-03 16:22:10.929303: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 24)]                 0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 160)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 24, 200)              422200    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 160, 200)             422200    ['input_2[0][0]']             
                                                                                              

2023-07-03 16:22:12.694249: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-03 16:22:12.981582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-03 16:22:13.114919: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_19/output/_22'
2023-07-03 16:2

 1/13 [=>............................] - ETA: 27s - loss: 7.6550 - accuracy: 0.0000e+00

2023-07-03 16:22:13.805952: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


13/13 [==============================] - 3s 78ms/step - loss: 7.5488 - accuracy: 0.0342
Epoch 2/10
13/13 [==============================] - 1s 73ms/step - loss: 6.6921 - accuracy: 0.0370
Epoch 3/10
13/13 [==============================] - 1s 67ms/step - loss: 6.4661 - accuracy: 0.0494
Epoch 4/10
13/13 [==============================] - 1s 69ms/step - loss: 6.3973 - accuracy: 0.0538
Epoch 5/10
13/13 [==============================] - 1s 66ms/step - loss: 6.3424 - accuracy: 0.0615
Epoch 6/10
13/13 [==============================] - 1s 63ms/step - loss: 6.3010 - accuracy: 0.0730
Epoch 7/10
13/13 [==============================] - 1s 65ms/step - loss: 6.2350 - accuracy: 0.0813
Epoch 8/10
13/13 [==============================] - 1s 63ms/step - loss: 6.1695 - accuracy: 0.0803
Epoch 9/10
13/13 [==============================] - 1s 65ms/step - loss: 6.0659 - accuracy: 0.0819
Epoch 10/10
13/13 [==============================] - 1s 65ms/step - loss: 5.9905 - accuracy: 0.0817


In [4]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [10]:
def inference():
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    decoder_state_input_h = tf.keras.layers.Input(shape=(200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(200 ,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs] + decoder_states)
    return encoder_model , decoder_model
def preprocess_input(input_sentence):
    tokens = re.sub('[^a-zA-Z]', ' ', input_sentence).lower().split()
    # tokens = tokenize(input_sentence)
    print(tokens)
    tokens_list = []
    for word in tokens:
        tokens_list.append(tokenizer.word_index[word])
    return preprocessing.sequence.pad_sequences([tokens_list] , maxlen=maxlen_questions , padding='post')

In [11]:
enc_model , dec_model = inference()  
model.save('test—300epoch')

INFO:tensorflow:Assets written to: test—300epoch/assets


INFO:tensorflow:Assets written to: test—300epoch/assets


In [12]:
tests = ['What is Python programming language in Picsart Academy?', 'What is C++?', 'In which parts of the learning model at Picsart Academy is JavaScript studied',
         'How many months last the course in Picsart Academy?', 'What is Sandbox at Picsart Academy', 'What courses there are in Level Up Bootcamp']

for i in range(5):
    states_values = enc_model.predict(preprocess_input(tests[i]))
    empty_target_seq = np.zeros((1 , 1))
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([empty_target_seq] + states_values)
        sampled_word_index = np.argmax(dec_outputs[0, -1, :])
        sampled_word = None
        
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += f' {word}'
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros((1 , 1))  
        empty_target_seq[0 , 0] = sampled_word_index
        states_values = [h , c] 
    print(f'Human: {tests[i]}')
    print()
    decoded_translation = decoded_translation.split(' end')[0]
    print(f'Bot: {decoded_translation}')
    print('-'*25)

['what', 'is', 'python', 'programming', 'language', 'in', 'picsart', 'academy']


2023-07-08 16:51:42.925848: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-08 16:51:43.045354: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 386ms/step


2023-07-08 16:51:43.413430: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-08 16:51:43.500324: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 12ms/step
Human: What is Python programming language in Picsart Academy?

Bot:  the only person you are destined to become is the person you decide to be ralph waldo emerson
-------------------------
['what', 'is', 'c']
1/1 [==============================] - 0s 14ms/step
Human: What is C++?

Bot:  the only person you are destined to become is the person you decide to be ralph waldo emerson
-------------------------
['in', 'which', 'parts', 'of', 'the', 'learning', 'model', 'at', 'picsart', 'academy', 'is', 'javascript', 'studied']
1/1 [==============================] - 0s 14ms/step
Human: In which parts of the learning model at Picsart Academy is JavaScript studied

Bot:  the only person you are destined to become is the person you decide to be ralph waldo emerson
-------------------------
['how', 'many', 'months', 'last', 'the', 'course', 'in', 'picsart', 'academy']
1/1 [==============================] - 0s 13ms/step
Human: How many months las